In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Swahili News Classification/Test.csv')
test_content_list = test['content'].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("flax-community/bert-swahili-news-classification", model_max_length=256)
model = AutoModelForSequenceClassification.from_pretrained("flax-community/bert-swahili-news-classification")

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
tokens = tokenizer(test_content_list, padding=True, truncation=True, return_tensors='pt').to(device)

model.eval()
with torch.no_grad():
    outputs = model(**tokens)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
probabilities_cpu = probabilities.cpu().numpy()

In [ ]:
columns = ['Biashara', 'Kitaifa', 'michezo', 'Kimataifa', 'Burudani', 'afya-health']

df = pd.DataFrame(probabilities_cpu, columns=columns)

In [ ]:
df['swahili_id'] = test['swahili_id']
new_order = ['swahili_id', 'Kitaifa', 'michezo', 'Biashara', 'Kimataifa', 'Burudani', 'afya-health']
df = df[new_order]

df = df.drop(['afya-health'], axis=1)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Swahili News Classification/submission.csv', index=False)
pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Swahili News Classification/submission.csv')

,swahili_id,Kitaifa,michezo,Biashara,Kimataifa,Burudani
0,001dd47ac202d9db6624a5ff734a5e7dddafeaf2,0.998010,0.000469,0.000419,0.000247,0.000031
1,0043d97f7690e9bc02f0ed8bb2b260d1d44bad92,0.019478,0.978081,0.000628,0.000545,0.000975
2,00579c2307b5c11003d21c40c3ecff5e922c3fd8,0.520223,0.000452,0.475977,0.002829,0.000251
3,00868eeee349e286303706ef0ffd851f39708d37,0.958453,0.001199,0.000172,0.002239,0.000409
4,00a5cb12d3058dcf2e42f277eee599992db32412,0.955126,0.000560,0.043873,0.000182,0.000059
...,...,...,...,...,...,...
1283,feb4b0ae88524c9cee3e50f2301d84a235f3c607,0.997636,0.000195,0.000540,0.000519,0.000029
1284,fecf3f14f47237e02721ed4baa4eb6c11abd239f,0.000275,0.999078,0.000283,0.000098,0.000227
1285,feed09e13586d12139cd59bc20996a4d29706606,0.157479,0.838159,0.000386,0.001225,0.002154
1286,ffc0bca6fde8fa0cce1ac3a7b0d746603d441bf8,0.015429,0.983187,0.000100,0.000323,0.000870
